In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

In [2]:
smileon_full = pd.read_csv('../data2/SmileOn11.28.2020.csv', skiprows=2, encoding='ISO-8859-1')

C:\Users\kkosf\anaconda3\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (32,36,53,59,88,89,127,141,144,156,157,158,159) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# create a new dataframe with fewer columns:

smileon_full['CallDateAndTimeStart'] = pd.to_datetime(smileon_full['CallDateAndTimeStart'])
smileon_full['date'] = smileon_full['CallDateAndTimeStart'].dt.date
smileon_full['time'] = smileon_full['CallDateAndTimeStart'].dt.time

smileon = smileon_full[['CallReportNum', 'ReportVersion', 'date', 'CallerNum', 'CountyName', 'PostalCode', 
                        'SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?',
                        'SMILE ON 60+ Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply)',
                        'SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?',
                        'SMILE ON 60+ Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?',
                        'SMILE ON 60+ Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?',
                        'SMILE ON 60+ Screening - How often do you brush your teeth?', 
                        'SMILE ON 60+ Screening - Need for Periodontal Care',
                        'SMILE ON 60+ Screening - Root Fragments', 
                        'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions',
                        'SMILE ON 60+ Screening - Untreated Decay', 
                        'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?', 
                        'SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)'
                       ]]

#and rename the columns to make things easier:

smileon = smileon.rename(columns = {'SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?': 'Last_Dentist_Visit',
                                    'SMILE ON 60+ Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply)' : 'Barriers',
                                    'SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?' : 'Eating_Limitations',
                                    'SMILE ON 60+ Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?' : 'Appearance_QOL',
                                    'SMILE ON 60+ Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?' : 'Condition',
                                    'SMILE ON 60+ Screening - How often do you brush your teeth?' : 'Brush_Frequency', 
                                    'SMILE ON 60+ Screening - Need for Periodontal Care' : 'Need_For_Care',
                                    'SMILE ON 60+ Screening - Root Fragments' : 'Root_Fragments', 
                                    'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions' : 'Lesions',
                                    'SMILE ON 60+ Screening - Untreated Decay' : 'Decay', 
                                    'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?' : 'Tx_Plan_Complete', 
                                    'SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)' :
                                    'Function_Restored'
                                   })

smileon = smileon.loc[~smileon['CallerNum'].isin(['-1', '-2'])]

smileon.head()

,CallReportNum,ReportVersion,date,CallerNum,CountyName,PostalCode,Last_Dentist_Visit,Barriers,Eating_Limitations,Appearance_QOL,Condition,Brush_Frequency,Need_For_Care,Root_Fragments,Lesions,Decay,Tx_Plan_Complete,Function_Restored
0,81381827,SMILE ON 60+ Registration,2020-11-25,3859874,Robertson,37172.0,3 to 5 years,Transportation; Dental cost; No insurance,No,Yes,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81379177,SMILE ON 60+ Registration,2020-11-25,3859755,Davidson,37212.0,3 to 5 years,Dental cost; No insurance,No,No,Very Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,81377435,SMILE ON 60+ Clinical,2020-11-25,2518680,Hawkins,37857.0,Less than 12 months,Dental cost,No,No,Good,NaN,NaN,NaN,NaN,NaN,Yes,Yes
3,81375215,SMILE ON 60+ Clinical,2020-11-25,3231923,Shelby,38104.0,Less than 12 months,Transportation; Dental cost; No insurance,Yes,Yes,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,81371535,SMILE ON 60+ Clinical,2020-11-25,3605119,Sullivan,37664.0,Less than 12 months,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN


In [4]:
smileon.isnull().sum()

CallReportNum             0
ReportVersion             0
date                      0
CallerNum                 0
CountyName              314
PostalCode             2569
Last_Dentist_Visit     2740
Barriers               5350
Eating_Limitations     6206
Appearance_QOL         6730
Condition              6165
Brush_Frequency       35388
Need_For_Care         35524
Root_Fragments        35296
Lesions               35590
Decay                 35130
Tx_Plan_Complete      22418
Function_Restored     25680
dtype: int64

In [5]:
smileon.CallerNum.nunique()

8706

In [6]:
smileon.ReportVersion.value_counts()

SMILE ON 60+ Clinical        27357
SMILE ON 60+ Registration    12595
Name: ReportVersion, dtype: int64

In [7]:
#separate out registration vs clinical:

reg_only = smileon[smileon['ReportVersion'] == 'SMILE ON 60+ Registration'] 
print(reg_only.shape)

clinic_only =  smileon[smileon['ReportVersion'] == 'SMILE ON 60+ Clinical'] 
print(clinic_only.shape)
clinic_only.head()

(12595, 18)
(27357, 18)


,CallReportNum,ReportVersion,date,CallerNum,CountyName,PostalCode,Last_Dentist_Visit,Barriers,Eating_Limitations,Appearance_QOL,Condition,Brush_Frequency,Need_For_Care,Root_Fragments,Lesions,Decay,Tx_Plan_Complete,Function_Restored
2,81377435,SMILE ON 60+ Clinical,2020-11-25,2518680,Hawkins,37857.0,Less than 12 months,Dental cost,No,No,Good,NaN,NaN,NaN,NaN,NaN,Yes,Yes
3,81375215,SMILE ON 60+ Clinical,2020-11-25,3231923,Shelby,38104.0,Less than 12 months,Transportation; Dental cost; No insurance,Yes,Yes,Poor,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,81371535,SMILE ON 60+ Clinical,2020-11-25,3605119,Sullivan,37664.0,Less than 12 months,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN
6,81371527,SMILE ON 60+ Clinical,2020-11-25,3859184,Wilson,37087.0,1 to 2 years,Dental cost,No,Yes,Fair,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,81371481,SMILE ON 60+ Clinical,2020-11-25,3746643,Hawkins,37642.0,More than 5 years,Dental cost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN


In [8]:
clinic_only.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27357 entries, 2 to 40029
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CallReportNum       27357 non-null  int64  
 1   ReportVersion       27357 non-null  object 
 2   date                27357 non-null  object 
 3   CallerNum           27357 non-null  int64  
 4   CountyName          27177 non-null  object 
 5   PostalCode          25680 non-null  float64
 6   Last_Dentist_Visit  25527 non-null  object 
 7   Barriers            23954 non-null  object 
 8   Eating_Limitations  23237 non-null  object 
 9   Appearance_QOL      22898 non-null  object 
 10  Condition           23244 non-null  object 
 11  Brush_Frequency     4564 non-null   object 
 12  Need_For_Care       4428 non-null   object 
 13  Root_Fragments      4656 non-null   object 
 14  Lesions             4362 non-null   object 
 15  Decay               4822 non-null   object 
 16  Tx_P

In [9]:
reg_only.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12595 entries, 0 to 40028
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CallReportNum       12595 non-null  int64  
 1   ReportVersion       12595 non-null  object 
 2   date                12595 non-null  object 
 3   CallerNum           12595 non-null  int64  
 4   CountyName          12461 non-null  object 
 5   PostalCode          11703 non-null  float64
 6   Last_Dentist_Visit  11685 non-null  object 
 7   Barriers            10648 non-null  object 
 8   Eating_Limitations  10509 non-null  object 
 9   Appearance_QOL      10324 non-null  object 
 10  Condition           10543 non-null  object 
 11  Brush_Frequency     0 non-null      object 
 12  Need_For_Care       0 non-null      object 
 13  Root_Fragments      0 non-null      object 
 14  Lesions             0 non-null      object 
 15  Decay               0 non-null      object 
 16  Tx_P

In [10]:
Eating_Limitations = smileon[smileon['Eating_Limitations'] == 'Yes'] 
Eating_Limitations.CallerNum.nunique()

4151

In [69]:
Eating_Limitations_fx = Eating_Limitations[Eating_Limitations['Function_Restored'] == 'Yes'] 
Eating_Limitations_fx.CallerNum.nunique()

1450

In [70]:
1450/4151 #35% of people that reported eating limitations have had function restored

0.3493134184533847

In [67]:
Eating_Limitations_tx = Eating_Limitations[Eating_Limitations['Tx_Plan_Complete'] == 'Yes'] 
Eating_Limitations_tx.CallerNum.nunique()

1297

In [68]:
1297/4151 #31% of people that reported eating limitations have completed a treatment plan

0.3124548301614069

In [15]:
Appearance_QOL = smileon[smileon['Appearance_QOL'] == 'Yes'] 
Appearance_QOL.CallerNum.nunique()

4761

In [65]:
Appearance_QOL_fx = Appearance_QOL[Appearance_QOL['Function_Restored'] == 'Yes'] 
Appearance_QOL_fx.CallerNum.nunique()

1780

In [66]:
1780/4761 #37% of people that reported appearance/QOL issues have had function restored

0.3738710354967444

In [63]:
Appearance_QOL_tx = Appearance_QOL[Appearance_QOL['Tx_Plan_Complete'] == 'Yes'] 
Appearance_QOL_tx.CallerNum.nunique()

1565

In [64]:
1565/4761 #32% of people that reported appearance/QOL issues have completed a treatment plan

0.32871245536651966

In [20]:
smileon.Condition.value_counts()

Poor         15687
Fair         11352
Good          4716
Dont know     1354
Very Good      678
Name: Condition, dtype: int64

In [21]:
Poor_condition = smileon[smileon['Condition'] == 'Poor']
Poor_condition.CallerNum.nunique()

3264

In [48]:
Poor_condition_fx = Poor_condition[Poor_condition['Function_Restored'] == 'Yes'] 
Poor_condition_fx.CallerNum.nunique()

1081

In [51]:
1081/3264 #33% of people that reported poor mouth condition have had function restored

0.33118872549019607

In [49]:
Poor_condition_fx = Poor_condition[Poor_condition['Tx_Plan_Complete'] == 'Yes'] 
Poor_condition_fx.CallerNum.nunique()

948

In [50]:
948/3264 #29% of people that reported poor mouth condition have completed a treatment plan

0.29044117647058826

In [26]:
Fair_condition = smileon[smileon['Condition'] == 'Fair']
Fair_condition.CallerNum.nunique()

2456

In [27]:
Fair_condition_fx = Fair_condition[Fair_condition['Function_Restored'] == 'Yes'] 
Fair_condition_fx.CallerNum.nunique()

1028

In [42]:
1028/2456 #42% of people that reported fair mouth condition have had function restored

0.4185667752442997

In [44]:
Fair_condition_tx = Fair_condition[Fair_condition['Tx_Plan_Complete'] == 'Yes'] 
Fair_condition_tx.CallerNum.nunique()

890

In [45]:
890/2456 #36% of people that reported poor mouth condition have completed a treatment plan

0.36237785016286644

In [72]:
smileon.Last_Dentist_Visit.value_counts()

Less than 12 months    16750
More than 5 years       8523
1 to 2 years            7076
3 to 5 years            4863
Name: Last_Dentist_Visit, dtype: int64

In [32]:
five_plus = smileon[smileon['Last_Dentist_Visit'] == 'More than 5 years'] 
five_plus.CallerNum.nunique()

2075

In [34]:
five_plus_fx = five_plus[five_plus['Function_Restored'] == 'Yes'] 
five_plus_fx.CallerNum.nunique()

573

In [38]:
573/2075 #28% of people that haven't been to the dentist in 5 years have function restored

0.276144578313253

In [40]:
five_plus_tx = five_plus[five_plus['Tx_Plan_Complete'] == 'Yes'] 
five_plus_tx.CallerNum.nunique()

532

In [41]:
532/2075 #26% of people that haven't been to the dentist in 5 years have completed a treatment plan

0.2563855421686747